### *NOTE - Please refer to project report pdf & queries sql file for more details*

# LIBRARIES

In [30]:
import json
import csv
import pandas as pd
import datetime
import sqlite3

### Parameters: 

In [31]:
dateTimeObj  = datetime.datetime.now()
timestampStr = dateTimeObj.strftime("%Y-%m-%d %H:%M:%S")
db = "bungalow.db"
print('Current Timestamp: ', timestampStr)
print('Database: ',db)
#
#---------------------------------#
rent_data_path = '/Users/naveenm/Downloads/takehome-data-eng-main/sample-data/zillow-rental-data.json'
#---------------------------------#
sale_data_path = '/Users/naveenm/Downloads/takehome-data-eng-main/sample-data/mls-sale-data.csv'
#

Current Timestamp:  2021-07-19 23:30:02
Database:  bungalow.db


# Stage 1 - Extract, Transform & Load

## Load from source Zillow File (Rent data): 

In [32]:
rent_df = pd.read_json(rent_data_path)
rent_df.shape

(44339, 25)

### Expand data column in JSON source file to read useful info:

In [33]:
df2     = pd.json_normalize(rent_df['data'])
fields  = ["home_status","zillow_id","home_type","year_built"]
df2     = df2[fields]
rent_df = rent_df.merge(df2,left_index=True,right_index=True)
rent_df.shape

(44339, 29)

# Load from source MLS File (Sale data): 

In [39]:
sale_df = pd.read_csv(sale_data_path)
sale_df.shape

(41042, 28)

In [40]:
### Rename column names for convenience and replace special characters

In [41]:
rent_df.columns = rent_df.columns.str.replace(' ','_').str.lower()
#---------------------------------# 
sale_df = sale_df.rename(columns={'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)': 'URL','MLS#':'MLS_ID'})
sale_df.columns = sale_df.columns.str.replace(' ','_').str.lower()

### Remove duplicate records, retain the last occurrence

In [42]:
sale_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41042 entries, 0 to 41041
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   sale_type                   41042 non-null  object 
 1   sold_date                   2 non-null      object 
 2   property_type               41042 non-null  object 
 3   address                     40692 non-null  object 
 4   city                        41010 non-null  object 
 5   state_or_province           41038 non-null  object 
 6   zip_or_postal_code          41035 non-null  object 
 7   price                       41041 non-null  float64
 8   beds                        40349 non-null  float64
 9   baths                       40089 non-null  float64
 10  location                    40258 non-null  object 
 11  square_feet                 36988 non-null  float64
 12  lot_size                    37835 non-null  float64
 13  year_built                  398

In [43]:
rent_df = rent_df.drop_duplicates(subset = ['id','url'], keep='last')
#---------------------------------#
sale_df = sale_df.drop_duplicates(subset = ['url','mls_id'], keep='last')

### Modify a few columns on both rent & sale datasets for consistency across the Database

In [44]:
rent_df['region'].replace({'Alabama':'AL','Alaska':'AK','Arizona':'AZ',
        'Arkansas':'AR','California':'CA','Colorado':'CO','Connecticut':'CT',
        'Delaware':'DE','District of Columbia':'DC','Florida':'FL','Georgia':'GA',
        'Hawaii':'HI','Idaho':'ID','Illinois':'IL','Indiana':'IN',
        'Iowa':'IA','Kansas':'KS','Kentucky':'KY','Louisiana':'LA',
        'Maine':'ME','Maryland':'MD','Massachusetts':'MA','Michigan':'MI',
        'Minnesota':'MN','Mississippi':'MS','Missouri':'MO','Montana':'MT',
        'Nebraska':'NE','Nevada':'NV','New Hampshire':'NH','New Jersey':'NJ',
        'New Mexico':'NM','New York':'NY','North Carolina':'NC','North Dakota':'ND',
        'Ohio':'OH','Oklahoma':'OK','Oregon':'OR','Pennsylvania':'PA',
        'Rhode Island':'RI','South Carolina':'SC','South Dakota':'SD','Tennessee':'TN',
        'Texas':'TX','Utah':'UT','Vermont':'VT','Virginia':'VA','Washington':'WA','West Virginia':'WV',
        'Wisconsin':'WI','Wyoming':'WY'}, inplace=True)
rent_df['market']      = rent_df.market.str.replace('-', '')
rent_df['home_status'] = rent_df.home_status.str.replace('FOR_RENT','ForRent')
rent_df['city']        = rent_df['city'].str.lower()
rent_df['data']        = rent_df['data'].astype('str')
#---------------------------------#
sale_df['state_or_province'].replace({'Alabama':'AL','Alaska':'AK','Arizona':'AZ',
        'Arkansas':'AR','California':'CA','Colorado':'CO','Connecticut':'CT',
        'Delaware':'DE','District of Columbia':'DC','Florida':'FL','Georgia':'GA',
        'Hawaii':'HI','Idaho':'ID','Illinois':'IL','Indiana':'IN',
        'Iowa':'IA','Kansas':'KS','Kentucky':'KY','Louisiana':'LA',
        'Maine':'ME','Maryland':'MD','Massachusetts':'MA','Michigan':'MI',
        'Minnesota':'MN','Mississippi':'MS','Missouri':'MO','Montana':'MT',
        'Nebraska':'NE','Nevada':'NV','New Hampshire':'NH','New Jersey':'NJ',
        'New Mexico':'NM','New York':'NY','North Carolina':'NC','North Dakota':'ND',
        'Ohio':'OH','Oklahoma':'OK','Oregon':'OR','Pennsylvania':'PA',
        'Rhode Island':'RI','South Carolina':'SC',
        'South Dakota':'SD','Tennessee':'TN','Texas':'TX','Utah':'UT',
        'Vermont':'VT','Virginia':'VA','Washington':'WA','West Virginia':'WV',
        'Wisconsin':'WI','Wyoming':'WY'}, inplace=True)
sale_df['market'] = sale_df.market.str.replace('-', '')
sale_df['city']   = sale_df['city'].str.lower()

### Adding Load timestamp before writing to DB:

In [45]:
rent_df["load_ts"] = timestampStr
#---------------------------------#
sale_df["load_ts"] = timestampStr

### Create DB Connection:

In [46]:
conn = sqlite3.connect(db)
print(conn)

# Load to DB:
#### Note - if_exists to be changed to 'append' for incremental load.
#### Master Tables have source data with Minor transformations

In [47]:
sale_df.to_sql('sale_df_master', conn, if_exists='replace', index=False)
#---------------------------------#
rent_df.to_sql('rent_df_master', conn, if_exists='replace', index=False)


### validation: read counts from DB & check a few rows

In [48]:
pd.read_sql('select count(*) from sale_df_master', conn)

,count(*)
0,39452


In [49]:
pd.read_sql('select count(*) from rent_df_master', conn)

,count(*)
0,44339


In [50]:
pd.read_sql('select * from sale_df_master limit 5', conn)

,sale_type,sold_date,property_type,address,city,state_or_province,zip_or_postal_code,price,beds,baths,...,next_open_house_end_time,url,source,mls_id,favorite,interested,latitude,longitude,market,load_ts
0,MLS Listing,None,Single Family Residential,129 W Upsal St,philadelphia,PA,19119,839000.0,6.0,3.5,...,June-26-2021 02:00 PM,http://www.redfin.com/PA/Philadelphia/129-W-Up...,BRIGHT MLS,PAPH2003206,N,Y,40.045757,-75.186434,philadelphia,2021-07-19 23:30:02
1,MLS Listing,None,Multi-Family (2-4 Unit),4236 Otter St,philadelphia,PA,19104,300000.0,6.0,3.0,...,None,http://www.redfin.com/PA/Philadelphia/4236-Ott...,BRIGHT MLS,PAPH1020452,N,Y,39.970768,-75.209881,philadelphia,2021-07-19 23:30:02
2,MLS Listing,None,Multi-Family (2-4 Unit),1326 N 7th St,philadelphia,PA,19122,575000.0,5.0,3.5,...,None,http://www.redfin.com/PA/Philadelphia/1326-N-7...,BRIGHT MLS,PAPH1005860,N,Y,39.972575,-75.147937,philadelphia,2021-07-19 23:30:02
3,MLS Listing,None,Multi-Family (2-4 Unit),4840 E Roosevelt Blvd,philadelphia,PA,19124,249900.0,NaN,NaN,...,None,http://www.redfin.com/PA/Philadelphia/4840-E-R...,BRIGHT MLS,PAPH999802,N,Y,40.026797,-75.095321,philadelphia,2021-07-19 23:30:02
4,MLS Listing,None,Multi-Family (2-4 Unit),1806 W Berks St,philadelphia,PA,19121,824900.0,NaN,NaN,...,None,http://www.redfin.com/PA/Philadelphia/1806-W-B...,BRIGHT MLS,PAPH2003774,N,Y,39.982389,-75.164274,philadelphia,2021-07-19 23:30:02


In [51]:
pd.read_sql('select * from rent_df_master limit 5', conn)

,id,source,market,url,type,listing_type,property_id,price,sqft,beds,...,longitude,data,date_added,date_last_updated,date_last_seen,home_status,zillow_id,home_type,year_built,load_ts
0,2943858,zillow,newyork,https://www.zillow.com/homedetails/126-W-25th-...,supply,multi_bed,be03a5a8-8d07-4666-8608-9aa1cf3e4d25,6995,1500.0,4.0,...,-73.993150,"{'city': 'New York', 'link': 'https://www.zill...",2021-07-04T04:59:55.347407+00:00,2021-07-04T04:59:55.351266+00:00,2021-07-04T04:59:55.351266+00:00,ForRent,2113894529,Apartment,None,2021-07-19 23:30:02
1,2943855,zillow,newyork,https://www.zillow.com/homedetails/348-E-66th-...,supply,multi_bed,14e82680-e585-4c40-b51c-e82fc731744b,2400,NaN,2.0,...,-73.959470,"{'city': 'New York', 'link': 'https://www.zill...",2021-07-04T04:43:07.330851+00:00,2021-07-04T04:43:07.334958+00:00,2021-07-04T04:43:07.334958+00:00,ForRent,2069772688,Apartment,None,2021-07-19 23:30:02
2,2943853,zillow,newyork,https://www.zillow.com/homedetails/205-W-15th-...,supply,multi_bed,097fad53-1b36-4dbd-9a83-eaf341c4c8cb,4750,750.0,2.0,...,-73.999855,"{'city': 'New York', 'link': 'https://www.zill...",2021-07-04T04:37:44.101649+00:00,2021-07-04T04:37:44.105638+00:00,2021-07-04T04:37:44.105638+00:00,ForRent,2096425072,Apartment,None,2021-07-19 23:30:02
3,2943816,zillow,newyork,https://www.zillow.com/homedetails/54-E-Brinke...,supply,multi_bed,9030684f-1d39-4068-a28d-5ec979fc8947,2590,3422.0,3.0,...,-73.996326,"{'city': 'Palisades Park', 'link': 'https://ww...",2021-07-04T04:33:19.255868+00:00,2021-07-04T04:33:19.259817+00:00,2021-07-04T04:33:19.259817+00:00,ForRent,37988522,Townhouse,None,2021-07-19 23:30:02
4,2943798,zillow,newyork,https://www.zillow.com/homedetails/44-Tompkins...,supply,multi_bed,fff53774-9548-4095-9c46-e8e71cb86aef,2099,875.0,3.0,...,-73.947174,"{'city': 'Brooklyn', 'link': 'https://www.zill...",2021-07-04T04:14:05.310886+00:00,2021-07-04T04:14:05.314815+00:00,2021-07-04T04:14:05.314815+00:00,ForRent,2077582448,Apartment,None,2021-07-19 23:30:02


# Stage 2 - Aggregate Source data from Database for insights enablement

## Create new Table with Aggregated data for data science/Data anaysis teams:

In [52]:
#
# Merge Data from Rent & Sales datasets:
#
merge_query = """SELECT * FROM (
                                SELECT mls_id     as id,
                                       'ForSale' as listing_type,
                                        source,
                                        status    as home_status,
                                        market    as market,
                                        property_type as home_type,
                                        address,
                                        city,
                                        zip_or_postal_code as postal,
                                        state_or_province as region,
                                        price,
                                        square_feet as sqft,
                                        beds,
                                        baths,
                                        year_built
                                FROM sale_df_master
                                UNION ALL
                                SELECT id,
                                       home_status as listing_type,
                                       source,
                                       'Active' as home_status,
                                       market,
                                       home_type,
                                       address,
                                       city,
                                       postal,
                                       region,
                                       price,
                                       sqft,
                                       beds,
                                       baths,
                                       year_built
                                FROM rent_df_master
                                    )"""

In [53]:
real_estate_df = pd.read_sql_query(merge_query, conn)

### Missing Data in the aggregated Data:

In [54]:
def missing_cols(df):
    '''prints out columns with the # of missing values'''
    total = 0
    for col in df.columns:
        missing_vals = df[col].isnull().sum()
        total += missing_vals
        if missing_vals != 0:
            print(f"{col} => {df[col].isnull().sum()}")
    
    if total == 0:
        print("no missing values left")
    else:
        print("\nTotal Missing values:%d"%total)
            


In [55]:
missing_cols(real_estate_df)

id => 1
home_type => 2
address => 335
city => 29
postal => 6
region => 4
price => 1
sqft => 8056
beds => 1073
baths => 949
year_built => 41281

Total Missing values:51737


### Duplicate Check in agg data before loading to the final DB Table:

In [56]:
x = real_estate_df.duplicated().any()
print(x)

False


In [57]:
real_estate_df_dup = real_estate_df[real_estate_df.duplicated()]
real_estate_df_dup

,id,listing_type,source,home_status,market,home_type,address,city,postal,region,price,sqft,beds,baths,year_built


### Adding timestamp before loading to DB:

In [58]:
dateTimeObj  = datetime.datetime.now()
timestampStr = dateTimeObj.strftime("%Y-%m-%d %H:%M:%S")
real_estate_df["load_ts"] = timestampStr

### Load Final Agg data to DB:

In [59]:
real_estate_df.to_sql('real_estate_gold', conn, if_exists='replace', index=False)
pd.read_sql('select count(*) from real_estate_gold', conn)

,count(*)
0,83791


### Commit DB & close Connection:

In [60]:
conn.commit()
conn.close()

### **NOTE - Please refer to project report pdf & queries sql file for more details **